In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the data
df = pd.read_csv(r"C:\Users\LENOVO\python datascience basics (hydrabad ds)\ASSIGNMENT DATA SCIENCE\Neural networks\Alphabets_data.csv")

# Explore
print("Shape:", df.shape)
print("Columns:", df.columns)
print("Missing values:\n", df.isnull().sum())
print("Class distribution:\n", df.iloc[:, -1].value_counts())  # assuming last column is the label

## Separate features and target
X = df.drop(columns=['letter'])  # all numeric features
y = df['letter']                 # target labels

# Ensure all feature columns are numeric (in case some are read as object)
X = X.apply(pd.to_numeric, errors='coerce')

# Drop any rows with invalid values (just to be safe)
valid_rows = ~X.isnull().any(axis=1)
X = X[valid_rows]
y = y[valid_rows]
# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_enc, test_size=0.2, stratify=y_enc, random_state=42)


Shape: (20000, 17)
Columns: Index(['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar',
       'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey',
       'yedge', 'yedgex'],
      dtype='object')
Missing values:
 letter    0
xbox      0
ybox      0
width     0
height    0
onpix     0
xbar      0
ybar      0
x2bar     0
y2bar     0
xybar     0
x2ybar    0
xy2bar    0
xedge     0
xedgey    0
yedge     0
yedgex    0
dtype: int64
Class distribution:
 yedgex
8     8047
7     3472
9     2358
6     1827
10    1578
5      992
11     868
4      478
12     137
3      130
13      49
2       30
1       17
14      13
15       2
0        2
Name: count, dtype: int64


In [6]:
invalid_rows = X_raw[~valid_rows]
print("Sample invalid rows:")
print(invalid_rows.head())


Sample invalid rows:
  letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  
0      10       8      0       8      0  
1       3       9      2       8      4  
2       3       7      3       7      3  
3       4      10      6      10      2  
4       5       9      1       7      5  


In [9]:
print("Shape of X:", X.shape)
print("Classes:", le.classes_)


Shape of X: (20000, 16)
Classes: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define basic ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y_train)), activation='softmax')  # one neuron per class
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_split=0.2, verbose=1)


ModuleNotFoundError: No module named 'tensorflow'

In [11]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


NameError: name 'history' is not defined

In [12]:
# Evaluate on test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy (Basic ANN): {accuracy:.4f}")


NameError: name 'model' is not defined

In [13]:
from sklearn.metrics import classification_report

y_pred = np.argmax(model.predict(X_test), axis=1)

print("Classification Report (Basic ANN):")
print(classification_report(y_test, y_pred, target_names=le.classes_))


NameError: name 'model' is not defined

In [14]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

# Function to create model (for search)
def create_model(hidden_layer_sizes=(64,), activation='relu', learning_rate=0.001):
    model = Sequential()
    model.add(Dense(hidden_layer_sizes[0], input_dim=X_train.shape[1], activation=activation))
    for units in hidden_layer_sizes[1:]:
        model.add(Dense(units, activation=activation))
    model.add(Dense(len(np.unique(y_train)), activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Wrap for sklearn
keras_clf = KerasClassifier(model=create_model, verbose=0)

# Param grid
param_dist = {
    "model__hidden_layer_sizes": [(64,), (128, 64), (128, 64, 32)],
    "model__activation": ['relu', 'tanh'],
    "model__learning_rate": [0.01, 0.001, 0.0001],
    "batch_size": [16, 32],
    "epochs": [50]
}

# Randomized Search
search = RandomizedSearchCV(estimator=keras_clf,
                            param_distributions=param_dist,
                            n_iter=5,  # or 10 for more thorough
                            cv=3,
                            verbose=2,
                            n_jobs=-1)

search.fit(X_train, y_train)


ModuleNotFoundError: No module named 'scikeras'

In [15]:
best_model = search.best_estimator_

# Predict
y_pred_best = best_model.predict(X_test)

print("Classification Report (Tuned Model):")
print(classification_report(y_test, y_pred_best, target_names=le.classes_))


NameError: name 'search' is not defined